In [2]:
import pywren
import pickle 
import time

def latency(n):
    from redis import Redis
    
    gateway_ip = '54.212.247.168'
    gateway_port = '8888'
    bind_port = '40000'
    redis = Redis(gateway_ip, int(gateway_port), bind_port=int(bind_port))
    
    random_num = np.random.rand(1, n) #generate n floats (each becomes 20B after serializing)
    rand_serial = pickle.dumps(random_num)
    t0 = time.time()
    for i in range(100):
        redis.set(i, rand_serial)
    t1 = time.time()
    for i in range(100):
        redis.get(i)
    t2 = time.time()
    return (n, (t1-t0)*10, (t2-t1)*10) # time in ms 

In [3]:
def latency_no_gateway(n):
    from redis import Redis
    
    gateway_ip = '54.212.247.168'
    gateway_port = '8888'
    bind_port = '40000'
    redis = Redis(gateway_ip, int(gateway_port), bind_port=int(bind_port))
    t3 = time.time()
    redis.set('k', 'v') # excluding the gateway roundtrip time 
    t4 = time.time()
    
    random_num = np.random.rand(1, n) #generate n floats (each becomes 20B after serializing)
    rand_serial = pickle.dumps(random_num)
    t0 = time.time()
    for i in range(100):
        redis.set(i, rand_serial)
    t1 = time.time()
    for i in range(100):
        redis.get(i)
    t2 = time.time()
    return (n, (t1-t0)*10, (t2-t1)*10, (t4-t3)*1000) # time in ms 

In [24]:
wrenexec = pywren.default_executor()

In [16]:
future = wrenexec.call_async(latency, 1)
future.result()

(1, 12.15857982635498, 4.605951309204102)

In [34]:
future = wrenexec.call_async(latency_no_gateway, 1)
future.result()

(1, 3.398590087890625, 3.3980488777160645, 851.0119915008545)

In [20]:
future = wrenexec.call_async(latency, 500)
future.result()

'''
[ec2-user@ip-172-31-23-239 LCP]$ python gateway_tcp.py 
Gateway started...
Adding server ('54.244.178.162', 58116)
7002
received client
Adding client ('54.244.178.162', 52394)
server need to be udpated
GATEWAY: Clients sent to servers

Adding server ('54.244.178.162', 57754)
7002
GATEWAY: Servers sent to clients
Exception in thread Thread-2:
Traceback (most recent call last):
  File "/usr/lib64/python2.7/threading.py", line 804, in __bootstrap_inner
    self.run()
  File "gateway_tcp.py", line 29, in run
    data = self.conn.recv(4096)
error: [Errno 104] Connection reset by peer
'''

ConnectionError: Error 32 while writing to socket. Broken pipe.